Script to run baseline:

```

python train.py link_prediction with \                                     
dataset='FB15k-237' \
inductive=True \
model='bert-dkrl' \
rel_model='transe' \
loss_fn='margin' \
regularizer=1e-2 \
max_len=32 \
num_negatives=64 \
lr=1e-4 \
use_scheduler=False \
batch_size=64 \
emb_batch_size=512 \
eval_batch_size=128 \
max_epochs=5 \
checkpoint=None \
use_cached_text=False

```

In [1]:
import os
import os.path as osp
import networkx as nx
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from sacred.run import Run
from logging import Logger
from sacred import Experiment
from sacred.observers import MongoObserver
from transformers import BertTokenizer, get_linear_schedule_with_warmup
from collections import defaultdict
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score

from data import CATEGORY_IDS
from data import GraphDataset, TextGraphDataset, GloVeTokenizer
#import models
#import utils

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/suparnaghanvatkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/suparnaghanvatkar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

In [23]:
def transe_score(heads, tails, rels):
    return -torch.norm(heads + rels - tails, dim=-1, p=1)

In [28]:
def margin_loss(pos_scores, neg_scores):
    loss = 1 - pos_scores + neg_scores
    loss[loss < 0] = 0
    return loss.mean()

def nll_loss(pos_scores, neg_scores):
    return (F.softplus(-pos_scores).mean() + F.softplus(neg_scores).mean()) / 2


def l2_regularization(heads, tails, rels):
    reg_loss = 0.0
    for tensor in (heads, tails, rels):
        reg_loss += torch.mean(tensor ** 2)

    return reg_loss / 3.0

In [24]:
class LinkPrediction(nn.Module):
    """A general link prediction model with a lookup table for relation
    embeddings."""
    def __init__(self, dim, rel_model, loss_fn, num_relations, regularizer):
        super().__init__()
        self.dim = dim
        self.normalize_embs = False
        self.regularizer = regularizer

        if rel_model == 'transe':
            self.score_fn = transe_score
            self.normalize_embs = True
        else:
            raise ValueError(f'Unknown relational model {rel_model}.')

        self.rel_emb = nn.Embedding(num_relations, self.dim)
        nn.init.xavier_uniform_(self.rel_emb.weight.data)

        if loss_fn == 'margin':
            self.loss_fn = margin_loss
        elif loss_fn == 'nll':
            self.loss_fn = nll_loss
        else:
            raise ValueError(f'Unkown loss function {loss_fn}')

    def encode(self, *args, **kwargs):
        ent_emb = self._encode_entity(*args, **kwargs)
        if self.normalize_embs:
            ent_emb = F.normalize(ent_emb, dim=-1)

        return ent_emb

    def _encode_entity(self, *args, **kwargs):
        raise NotImplementedError

    def forward(self, *args, **kwargs):
        raise NotImplementedError

    def compute_loss(self, ent_embs, rels, neg_idx):
        batch_size = ent_embs.shape[0]

        # Scores for positive samples
        rels = self.rel_emb(rels)
        heads, tails = torch.chunk(ent_embs, chunks=2, dim=1)
        pos_scores = self.score_fn(heads, tails, rels)

        if self.regularizer > 0:
            reg_loss = self.regularizer * l2_regularization(heads, tails, rels)
        else:
            reg_loss = 0

        # Scores for negative samples
        neg_embs = ent_embs.view(batch_size * 2, -1)[neg_idx]
        heads, tails = torch.chunk(neg_embs, chunks=2, dim=2)
        neg_scores = self.score_fn(heads.squeeze(), tails.squeeze(), rels)

        model_loss = self.loss_fn(pos_scores, neg_scores)
        return model_loss + reg_loss

In [19]:
class InductiveLinkPrediction(LinkPrediction):
    """Description-based Link Prediction (DLP)."""
    def _encode_entity(self, text_tok, text_mask):
        raise NotImplementedError

    def forward(self, text_tok, text_mask, rels=None, neg_idx=None):
        batch_size, _, num_text_tokens = text_tok.shape

        # Encode text into an entity representation from its description
        ent_embs = self.encode(text_tok.view(-1, num_text_tokens),
                               text_mask.view(-1, num_text_tokens))

        if rels is None and neg_idx is None:
            # Forward is being used to compute entity embeddings only
            out = ent_embs
        else:
            # Forward is being used to compute link prediction loss
            ent_embs = ent_embs.view(batch_size, 2, -1)
            out = self.compute_loss(ent_embs, rels, neg_idx)

        return out

In [20]:
class WordEmbeddingsLP(InductiveLinkPrediction):
    """Description encoder with pretrained embeddings, obtained from BERT or a
    specified tensor file.
    """
    def __init__(self, rel_model, loss_fn, num_relations, regularizer,
                 dim=None, encoder_name=None, embeddings=None):
        if not encoder_name and not embeddings:
            raise ValueError('Must provided one of encoder_name or embeddings')

        if encoder_name is not None:
            encoder = BertModel.from_pretrained(encoder_name)
            embeddings = encoder.embeddings.word_embeddings
        else:
            emb_tensor = torch.load(embeddings)
            num_embeddings, embedding_dim = emb_tensor.shape
            embeddings = nn.Embedding(num_embeddings, embedding_dim)
            embeddings.weight.data = emb_tensor

        if dim is None:
            dim = embeddings.embedding_dim

        super().__init__(dim, rel_model, loss_fn, num_relations, regularizer)

        self.embeddings = embeddings

    def _encode_entity(self, text_tok, text_mask):
        raise NotImplementedError

In [21]:
class DKRL(WordEmbeddingsLP):
    """Description-Embodied Knowledge Representation Learning (DKRL) with CNN
    encoder, after
    Zuo, Yukun, et al. "Representation learning of knowledge graphs with
    entity attributes and multimedia descriptions."
    """

    def __init__(self, dim, rel_model, loss_fn, num_relations, regularizer,
                 encoder_name=None, embeddings=None):
        super().__init__(rel_model, loss_fn, num_relations, regularizer,
                         dim, encoder_name, embeddings)

        emb_dim = self.embeddings.embedding_dim
        self.conv1 = nn.Conv1d(emb_dim, self.dim, kernel_size=2)
        self.conv2 = nn.Conv1d(self.dim, self.dim, kernel_size=2)

    def _encode_entity(self, text_tok, text_mask):
        if text_mask is None:
            text_mask = torch.ones_like(text_tok, dtype=torch.float)
        # Extract word embeddings and mask padding
        embs = self.embeddings(text_tok) * text_mask.unsqueeze(dim=-1)

        # Reshape to (N, C, L)
        embs = embs.transpose(1, 2)
        text_mask = text_mask.unsqueeze(1)

        # Pass through CNN, adding padding for valid convolutions
        # and masking outputs due to padding
        embs = F.pad(embs, [0, 1])
        embs = self.conv1(embs)
        embs = embs * text_mask
        if embs.shape[2] >= 4:
            kernel_size = 4
        elif embs.shape[2] == 1:
            kernel_size = 1
        else:
            kernel_size = 2
        embs = F.max_pool1d(embs, kernel_size=kernel_size)
        text_mask = F.max_pool1d(text_mask, kernel_size=kernel_size)
        embs = torch.tanh(embs)
        embs = F.pad(embs, [0, 1])
        embs = self.conv2(embs)
        lengths = torch.sum(text_mask, dim=-1)
        embs = torch.sum(embs * text_mask, dim=-1) / lengths
        embs = torch.tanh(embs)

        return embs

utils

In [7]:
import logging

In [14]:
def get_model(model, dim, rel_model, loss_fn, num_entities, num_relations,
              encoder_name, regularizer):
    if model == 'bert-dkrl':
        return DKRL(dim, rel_model, loss_fn, num_relations, regularizer,
                           encoder_name=encoder_name)
    elif model == 'glove-dkrl':
        return DKRL(dim, rel_model, loss_fn, num_relations, regularizer,
                           embeddings='data/glove/glove.6B.300d.pt')
    else:
        raise ValueError(f'Unkown model {model}')


def make_ent2idx(entities, max_ent_id):
    """Given a tensor with entity IDs, return a tensor indexed with
    an entity ID, containing the position of the entity.
    Empty positions are filled with -1.

    Example:
    > make_ent2idx(torch.tensor([4, 5, 0]))
    tensor([ 2, -1, -1, -1,  0,  1])
    """
    idx = torch.arange(entities.shape[0])
    ent2idx = torch.empty(max_ent_id + 1, dtype=torch.long).fill_(-1)
    ent2idx.scatter_(0, entities, idx)
    return ent2idx


def get_triple_filters(triples, graph, num_ents, ent2idx):
    """Given a set of triples, filter candidate entities that are valid
    substitutes of an entity in the triple at a given position (head or tail).
    For a particular triple, this allows to compute rankings for an entity of
    interest, against other entities in the graph that would actually be wrong
    substitutes.
    Results are returned as a mask array with a value of 1.0 for filtered
    entities, and 0.0 otherwise.

    Args:
        triples: Bx3 tensor of type torch.long, where B is the batch size,
            and each row contains a triple of the form (head, tail, rel)
        graph: nx.MultiDiGraph containing all edges used to filter candidates
        num_ents: int, number of candidate entities
        ent2idx: tensor, contains at index ent_id the index of the column for
            that entity in the output mask array
    """
    num_triples = triples.shape[0]
    heads_filter = torch.zeros((num_triples, num_ents), dtype=torch.bool)
    tails_filter = torch.zeros_like(heads_filter)

    triples = triples.tolist()
    for i, (head, tail, rel) in enumerate(triples):
        head_edges = graph.out_edges(head, data='weight')
        for (h, t, r) in head_edges:
            if r == rel and t != tail:
                ent_idx = ent2idx[t]
                if ent_idx != -1:
                    tails_filter[i, ent_idx] = True

        tail_edges = graph.in_edges(tail, data='weight')
        for (h, t, r) in tail_edges:
            if r == rel and h != head:
                ent_idx = ent2idx[h]
                if ent_idx != -1:
                    heads_filter[i, ent_idx] = True

    return heads_filter, tails_filter


def hit_at_k(predictions, ground_truth_idx, hit_positions):
    """Calculates mean number of hits@k. Higher values are ranked first.

    Args:
        predictions: BxN tensor of prediction values where B is batch size
            and N number of classes.
        ground_truth_idx: Bx1 tensor with index of ground truth class
        hit_positions: list, containing number of top K results to be
            considered as hits.

    Returns: list of float, of the same length as hit_positions, containing
        Hits@K score.
    """
    max_position = max(hit_positions)
    _, indices = predictions.topk(k=max_position)
    hits = []

    for position in hit_positions:
        idx_at_k = indices[:, :position]
        hits_at_k = (idx_at_k == ground_truth_idx).sum(dim=1).float().mean()
        hits.append(hits_at_k.item())

    return hits


def mrr(predictions, ground_truth_idx):
    """Calculates mean reciprocal rank (MRR) for given predictions
    and ground truth values. Higher values are ranked first.

    Args:
        predictions: BxN tensor of prediction values where B is batch size
            and N number of classes.
        ground_truth_idx: Bx1 tensor with index of ground truth class

    Returns: float, Mean reciprocal rank score
    """
    indices = predictions.argsort(descending=True)
    rankings = (indices == ground_truth_idx).nonzero()[:, 1].float() + 1.0
    return rankings.reciprocal()


def split_by_new_position(triples, mrr_values, new_entities):
    """Split MRR results by the position of new entity. Use to break down
    results for a triple where a new entity is at the head and the tail,
    at the head only, or the tail only.
    Since MRR is calculated by corrupting the head first, and then the head,
    the size of mrr_values should be twice the size of triples. The calculated
    MRR is then the average of the two cases.
    Args:
        triples: Bx3 tensor containing (head, tail, rel).
        mrr_values: 2B tensor, with first half containing MRR for corrupted
            triples at the head position, and second half at the tail position.
        new_entities: set, entities to be considered as new.
    Returns:
        mrr_by_position: tensor of 3 elements breaking down MRR by new entities
            at both positions, at head, and tail.
        mrr_pos_counts: tensor of 3 elements containing counts for each case.
    """
    mrr_by_position = torch.zeros(3, device=mrr_values.device)
    mrr_pos_counts = torch.zeros_like(mrr_by_position)
    num_triples = triples.shape[0]

    for i, (h, t, r) in enumerate(triples):
        head, tail = h.item(), t.item()
        mrr_val = (mrr_values[i] + mrr_values[i + num_triples]).item() / 2.0
        if head in new_entities and tail in new_entities:
            mrr_by_position[0] += mrr_val
            mrr_pos_counts[0] += 1.0
        elif head in new_entities:
            mrr_by_position[1] += mrr_val
            mrr_pos_counts[1] += 1.0
        elif tail in new_entities:
            mrr_by_position[2] += mrr_val
            mrr_pos_counts[2] += 1.0

    return mrr_by_position, mrr_pos_counts


def split_by_category(triples, mrr_values, rel_categories):
    mrr_by_category = torch.zeros([2, 4], device=mrr_values.device)
    mrr_cat_count = torch.zeros([1, 4], dtype=torch.float,
                                device=mrr_by_category.device)
    num_triples = triples.shape[0]

    for i, (h, t, r) in enumerate(triples):
        rel_category = rel_categories[r]

        mrr_val_head_pred = mrr_values[i]
        mrr_by_category[0, rel_category] += mrr_val_head_pred

        mrr_val_tail_pred = mrr_values[i + num_triples]
        mrr_by_category[1, rel_category] += mrr_val_tail_pred

        mrr_cat_count[0, rel_category] += 1

    return mrr_by_category, mrr_cat_count


def get_logger():
    """Get a default logger that includes a timestamp."""
    logger = logging.getLogger("")
    logger.handlers = []
    ch = logging.StreamHandler()
    str_fmt = '%(asctime)s - %(levelname)s - %(name)s - %(message)s'
    formatter = logging.Formatter(str_fmt, datefmt='%H:%M:%S')
    ch.setFormatter(formatter)
    logger.addHandler(ch)
    logger.setLevel('INFO')

    return logger


In [10]:
OUT_PATH = 'output/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/home/suparnaghanvatkar/anaconda3/envs/blp/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [11]:
def eval_link_prediction(model, triples_loader, text_dataset, entities,
                         epoch, emb_batch_size,
                         prefix='', max_num_batches=None,
                         filtering_graph=None, new_entities=None,
                         return_embeddings=False):
    compute_filtered = filtering_graph is not None
    mrr_by_position = torch.zeros(3, dtype=torch.float).to(device)
    mrr_pos_counts = torch.zeros_like(mrr_by_position)

    rel_categories = triples_loader.dataset.rel_categories.to(device)
    mrr_by_category = torch.zeros([2, 4], dtype=torch.float).to(device)
    mrr_cat_count = torch.zeros([1, 4], dtype=torch.float).to(device)

    hit_positions = [1, 3, 10]
    hits_at_k = {pos: 0.0 for pos in hit_positions}
    mrr_value = 0.0
    mrr_filt = 0.0
    hits_at_k_filt = {pos: 0.0 for pos in hit_positions}

    if device != torch.device('cpu'):
        model = model.module

    if isinstance(model, InductiveLinkPrediction):
        num_entities = entities.shape[0]
        if compute_filtered:
            max_ent_id = max(filtering_graph.nodes)
        else:
            max_ent_id = entities.max()
        ent2idx = make_ent2idx(entities, max_ent_id)
    else:
        # Transductive models have a lookup table of embeddings
        num_entities = model.ent_emb.num_embeddings
        ent2idx = torch.arange(num_entities)
        entities = ent2idx

    # Create embedding lookup table for evaluation
    ent_emb = torch.zeros((num_entities, model.dim), dtype=torch.float,
                          device=device)
    idx = 0
    num_iters = np.ceil(num_entities / emb_batch_size)
    iters_count = 0
    while idx < num_entities:
        # Get a batch of entity IDs and encode them
        batch_ents = entities[idx:idx + emb_batch_size]

        if isinstance(model, InductiveLinkPrediction):
            # Encode with entity descriptions
            data = text_dataset.get_entity_description(batch_ents)
            text_tok, text_mask, text_len = data
            batch_emb = model(text_tok.unsqueeze(1).to(device),
                              text_mask.unsqueeze(1).to(device))
        else:
            # Encode from lookup table
            batch_emb = model(batch_ents)

        ent_emb[idx:idx + batch_ents.shape[0]] = batch_emb

        iters_count += 1
        if iters_count % np.ceil(0.2 * num_iters) == 0:
            print(f'[{idx + batch_ents.shape[0]:,}/{num_entities:,}]')

        idx += emb_batch_size

    ent_emb = ent_emb.unsqueeze(0)

    batch_count = 0
    print('Computing metrics on set of triples')
    total = len(triples_loader) if max_num_batches is None else max_num_batches
    for i, triples in enumerate(triples_loader):
        if max_num_batches is not None and i == max_num_batches:
            break

        heads, tails, rels = torch.chunk(triples, chunks=3, dim=1)
        # Map entity IDs to positions in ent_emb
        heads = ent2idx[heads].to(device)
        tails = ent2idx[tails].to(device)

        assert heads.min() >= 0
        assert tails.min() >= 0

        # Embed triple
        head_embs = ent_emb.squeeze()[heads]
        tail_embs = ent_emb.squeeze()[tails]
        rel_embs = model.rel_emb(rels.to(device))

        # Score all possible heads and tails
        heads_predictions = model.score_fn(ent_emb, tail_embs, rel_embs)
        tails_predictions = model.score_fn(head_embs, ent_emb, rel_embs)

        pred_ents = torch.cat((heads_predictions, tails_predictions))
        true_ents = torch.cat((heads, tails))

        hits = hit_at_k(pred_ents, true_ents, hit_positions)
        for j, h in enumerate(hits):
            hits_at_k[hit_positions[j]] += h
        mrr_value += mrr(pred_ents, true_ents).mean().item()

        if compute_filtered:
            filters = get_triple_filters(triples, filtering_graph,
                                               num_entities, ent2idx)
            heads_filter, tails_filter = filters
            # Filter entities by assigning them the lowest score in the batch
            filter_mask = torch.cat((heads_filter, tails_filter)).to(device)
            pred_ents[filter_mask] = pred_ents.min() - 1.0
            hits_filt = hit_at_k(pred_ents, true_ents, hit_positions)
            for j, h in enumerate(hits_filt):
                hits_at_k_filt[hit_positions[j]] += h
            mrr_filt_per_triple = mrr(pred_ents, true_ents)
            mrr_filt += mrr_filt_per_triple.mean().item()

            if new_entities is not None:
                by_position = split_by_new_position(triples,
                                                          mrr_filt_per_triple,
                                                          new_entities)
                batch_mrr_by_position, batch_mrr_pos_counts = by_position
                mrr_by_position += batch_mrr_by_position
                mrr_pos_counts += batch_mrr_pos_counts

            if triples_loader.dataset.has_rel_categories:
                by_category = split_by_category(triples,
                                                      mrr_filt_per_triple,
                                                      rel_categories)
                batch_mrr_by_cat, batch_mrr_cat_count = by_category
                mrr_by_category += batch_mrr_by_cat
                mrr_cat_count += batch_mrr_cat_count

        batch_count += 1
        if (i + 1) % int(0.2 * total) == 0:
            print(f'[{i + 1:,}/{total:,}]')

    for hits_dict in (hits_at_k, hits_at_k_filt):
        for k in hits_dict:
            hits_dict[k] /= batch_count

    mrr_value = mrr_value / batch_count
    mrr_filt = mrr_filt / batch_count

    log_str = f'{prefix} mrr: {mrr_value:.4f}  '
    print(f'{prefix}_mrr', mrr_value, epoch)
    for k, value in hits_at_k.items():
        log_str += f'hits@{k}: {value:.4f}  '
        print(f'{prefix}_hits@{k}', value, epoch)

    if compute_filtered:
        log_str += f'mrr_filt: {mrr_filt:.4f}  '
        print(f'{prefix}_mrr_filt', mrr_filt, epoch)
        for k, value in hits_at_k_filt.items():
            log_str += f'hits@{k}_filt: {value:.4f}  '
            print(f'{prefix}_hits@{k}_filt', value, epoch)

    print(log_str)

    if new_entities is not None and compute_filtered:
        mrr_pos_counts[mrr_pos_counts < 1.0] = 1.0
        mrr_by_position = mrr_by_position / mrr_pos_counts
        log_str = ''
        for i, t in enumerate((f'{prefix}_mrr_filt_both_new',
                               f'{prefix}_mrr_filt_head_new',
                               f'{prefix}_mrr_filt_tail_new')):
            value = mrr_by_position[i].item()
            log_str += f'{t}: {value:.4f}  '
            print(t, value, epoch)
        print(log_str)

    if compute_filtered and triples_loader.dataset.has_rel_categories:
        mrr_cat_count[mrr_cat_count < 1.0] = 1.0
        mrr_by_category = mrr_by_category / mrr_cat_count

        for i, case in enumerate(['pred_head', 'pred_tail']):
            log_str = f'{case} '
            for cat, cat_id in CATEGORY_IDS.items():
                log_str += f'{cat}_mrr: {mrr_by_category[i, cat_id]:.4f}  '
            print(log_str)

    if return_embeddings:
        out = (mrr_value, ent_emb)
    else:
        out = (mrr_value, None)

    return out

In [12]:
def link_prediction(dataset, inductive, dim, model, rel_model, loss_fn,
                    encoder_name, regularizer, max_len, num_negatives, lr,
                    use_scheduler, batch_size, emb_batch_size, eval_batch_size,
                    max_epochs, checkpoint, use_cached_text):
    drop_stopwords = model in {'bert-bow', 'bert-dkrl',
                               'glove-bow', 'glove-dkrl'}

    prefix = 'ind-' if inductive and model != 'transductive' else ''
    triples_file = f'data/{dataset}/{prefix}train.tsv'

    if device != torch.device('cpu'):
        num_devices = torch.cuda.device_count()
        if batch_size % num_devices != 0:
            raise ValueError(f'Batch size ({batch_size}) must be a multiple of'
                             f' the number of CUDA devices ({num_devices})')
        print(f'CUDA devices used: {num_devices}')
    else:
        num_devices = 1
        print('Training on CPU')

    if model == 'transductive':
        train_data = GraphDataset(triples_file, num_negatives,
                                  write_maps_file=True,
                                  num_devices=num_devices)
    else:
        if model.startswith('bert') or model == 'blp':
            tokenizer = BertTokenizer.from_pretrained(encoder_name)
        else:
            tokenizer = GloVeTokenizer('data/glove/glove.6B.300d-maps.pt')

        train_data = TextGraphDataset(triples_file, num_negatives,
                                      max_len, tokenizer, drop_stopwords,
                                      write_maps_file=True,
                                      use_cached_text=use_cached_text,
                                      num_devices=num_devices)

    train_loader = DataLoader(train_data, batch_size, shuffle=True,
                              collate_fn=train_data.collate_fn,
                              num_workers=0, drop_last=True)

    train_eval_loader = DataLoader(train_data, eval_batch_size)

    valid_data = GraphDataset(f'data/{dataset}/{prefix}dev.tsv')
    valid_loader = DataLoader(valid_data, eval_batch_size)

    test_data = GraphDataset(f'data/{dataset}/{prefix}test.tsv')
    test_loader = DataLoader(test_data, eval_batch_size)

    # Build graph with all triples to compute filtered metrics
    if dataset != 'Wikidata5M':
        graph = nx.MultiDiGraph()
        all_triples = torch.cat((train_data.triples,
                                 valid_data.triples,
                                 test_data.triples))
        graph.add_weighted_edges_from(all_triples.tolist())

        train_ent = set(train_data.entities.tolist())
        train_val_ent = set(valid_data.entities.tolist()).union(train_ent)
        train_val_test_ent = set(test_data.entities.tolist()).union(train_val_ent)
        val_new_ents = train_val_ent.difference(train_ent)
        test_new_ents = train_val_test_ent.difference(train_val_ent)
    else:
        graph = None

        train_ent = set(train_data.entities.tolist())
        train_val_ent = set(valid_data.entities.tolist())
        train_val_test_ent = set(test_data.entities.tolist())
        val_new_ents = test_new_ents = None

    print('num_train_entities', len(train_ent))

    train_ent = torch.tensor(list(train_ent))
    train_val_ent = torch.tensor(list(train_val_ent))
    train_val_test_ent = torch.tensor(list(train_val_test_ent))

    model = get_model(model, dim, rel_model, loss_fn,
                            len(train_val_test_ent), train_data.num_rels,
                            encoder_name, regularizer)
    if checkpoint is not None:
        model.load_state_dict(torch.load(checkpoint, map_location='cpu'))

    if device != torch.device('cpu'):
        model = torch.nn.DataParallel(model).to(device)

    optimizer = Adam(model.parameters(), lr=lr)
    total_steps = len(train_loader) * max_epochs
    if use_scheduler:
        warmup = int(0.2 * total_steps)
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=warmup,
                                                    num_training_steps=total_steps)
    best_valid_mrr = 0.0
    checkpoint_file = osp.join(OUT_PATH, f'model-base.pt')
    for epoch in range(1, max_epochs + 1):
        train_loss = 0
        for step, data in enumerate(train_loader):
            loss = model(*data).mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if use_scheduler:
                scheduler.step()

            train_loss += loss.item()

            if step % int(0.05 * len(train_loader)) == 0:
                print(f'Epoch {epoch}/{max_epochs} '
                          f'[{step}/{len(train_loader)}]: {loss.item():.6f}')
                print('batch_loss', loss.item())

        print('train_loss', train_loss / len(train_loader), epoch)

        if dataset != 'Wikidata5M':
            print('Evaluating on sample of training set')
            eval_link_prediction(model, train_eval_loader, train_data, train_ent,
                                 epoch, emb_batch_size, prefix='train',
                                 max_num_batches=len(valid_loader))

        print('Evaluating on validation set')
        val_mrr, _ = eval_link_prediction(model, valid_loader, train_data,
                                          train_val_ent, epoch,
                                          emb_batch_size, prefix='valid')

        # Keep checkpoint of best performing model (based on raw MRR)
        if val_mrr > best_valid_mrr:
            best_valid_mrr = val_mrr
            torch.save(model.state_dict(), checkpoint_file)

    # Evaluate with best performing checkpoint
    if max_epochs > 0:
        model.load_state_dict(torch.load(checkpoint_file))

    if dataset == 'Wikidata5M':
        graph = nx.MultiDiGraph()
        graph.add_weighted_edges_from(valid_data.triples.tolist())

    print('Evaluating on validation set (with filtering)')
    eval_link_prediction(model, valid_loader, train_data, train_val_ent,
                         max_epochs + 1, emb_batch_size, prefix='valid',
                         filtering_graph=graph,
                         new_entities=val_new_ents)

    if dataset == 'Wikidata5M':
        graph = nx.MultiDiGraph()
        graph.add_weighted_edges_from(test_data.triples.tolist())

    print('Evaluating on test set')
    _, ent_emb = eval_link_prediction(model, test_loader, train_data,
                                      train_val_test_ent, max_epochs + 1,
                                      emb_batch_size, prefix='test',
                                      filtering_graph=graph,
                                      new_entities=test_new_ents,
                                      return_embeddings=True)

    # Save final entity embeddings obtained with trained encoder
    torch.save(ent_emb, osp.join(OUT_PATH, f'ent_emb-base.pt'))
    torch.save(train_val_test_ent, osp.join(OUT_PATH, f'ents-base.pt'))



In [29]:
link_prediction(dataset='FB15k-237', inductive=True, dim=128, model='bert-dkrl', rel_model='transe', loss_fn='margin', \
                    encoder_name='bert-base-cased', regularizer=1e-2, max_len=32, num_negatives=64, lr=1e-4, use_scheduler=False, batch_size=64, emb_batch_size=512, eval_batch_size=128,\
                    max_epochs=1, checkpoint=None, use_cached_text=None)

Training on CPU


Reading entity descriptions: 100%|██████████| 14541/14541 [00:30<00:00, 469.13it/s]


num_train_entities 11633
Epoch 1/1 [0/3360]: 0.993732
batch_loss 0.9937319159507751
Epoch 1/1 [168/3360]: 0.256638
batch_loss 0.2566378712654114
Epoch 1/1 [336/3360]: 0.159425
batch_loss 0.15942464768886566
Epoch 1/1 [504/3360]: 0.112708
batch_loss 0.11270785331726074
Epoch 1/1 [672/3360]: 0.138466
batch_loss 0.13846643269062042
Epoch 1/1 [840/3360]: 0.090077
batch_loss 0.09007653594017029
Epoch 1/1 [1008/3360]: 0.099623
batch_loss 0.09962254017591476
Epoch 1/1 [1176/3360]: 0.105886
batch_loss 0.10588645935058594
Epoch 1/1 [1344/3360]: 0.078954
batch_loss 0.07895425707101822
Epoch 1/1 [1512/3360]: 0.076769
batch_loss 0.07676875591278076
Epoch 1/1 [1680/3360]: 0.073372
batch_loss 0.07337197661399841
Epoch 1/1 [1848/3360]: 0.085313
batch_loss 0.08531321585178375
Epoch 1/1 [2016/3360]: 0.072368
batch_loss 0.07236793637275696
Epoch 1/1 [2184/3360]: 0.064796
batch_loss 0.06479629874229431
Epoch 1/1 [2352/3360]: 0.081583
batch_loss 0.08158264309167862
Epoch 1/1 [2520/3360]: 0.062315
batch_lo

TypeError: 'float' object is not callable